# Circuit cutting data plotting notebook for Perlmutter, lightning.gpu and Ray

In [ ]:
%matplotlib qt5

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from io import StringIO
from scipy.optimize import curve_fit, Bounds
plt.rcParams['text.usetex'] = True

Data taken directly from SLURM log-files.

In [ ]:
df_batched = pd.read_csv("./QAOA_QCUT_BATCHED_CUDA12/BATCHED.csv").sort_values('num_nodes')
df_ondemand = pd.read_csv("./QAOA_QCUT_ONDEMAND_CUDA12/ONDEMAND.csv").sort_values('num_nodes')

In [ ]:
df_batched

In [ ]:
df_ondemand

# Runtime data for different scheduling strategies

In [ ]:
cmap = matplotlib.colormaps['tab20']
colors = cmap.colors

In [ ]:
plt.figure()
plt.rcParams.update({'font.size': 16})
plt.plot(df_batched["num_nodes"], df_batched["frag_exec_time"], linewidth=3, label="batched", marker="o", color=colors[0], mec="k", mfc="w")
plt.plot(df_ondemand["num_nodes"], df_ondemand["frag_exec_time"], linewidth=3, label="on-demand", marker="o", color=colors[2], mec="k", mfc="w")
plt.yscale("log")
plt.xscale("log")
plt.xticks([2**i for i in range(len(df_ondemand["num_nodes"]))], labels=[f"${{{i}}}$" for i in df_ondemand["num_nodes"]])
plt.ylabel("$\\textrm{circuit exec. time (s)}$")
plt.xlabel("$\\textrm{nodes}$")

plt.legend()

In [ ]:
def time_per_cct(data):
    num_circuits = data["num_cut_circuits"][0]
    num_nodes =  data["num_nodes"] # [2**i for i in list((data.keys()))]
    num_gpus = data["num_gpus"] # [4*i for i in num_nodes] 
    t = np.array(data["frag_exec_time"])    #frag_exec_time
    t_per_cct = t / num_circuits
    return t_per_cct

In [ ]:
def func(x, a, b):
    "Linear fit function"
    return a * x + b

# Time per circuit based on GPU count

In [ ]:
plt.figure()

popt_batch, pcov_batch = curve_fit(func, np.array(df_batched["num_nodes"]),  1/time_per_cct(df_batched) )
popt_ondemand, pcov_ondemand = curve_fit(func, np.array(df_ondemand["num_nodes"]),  1/time_per_cct(df_ondemand) )
num_nodes = df_batched["num_nodes"]

plt.scatter([i for i in num_nodes], 1/time_per_cct(df_batched), label="$\\textrm{batched}$",  marker="o", color=colors[0], ec="k", zorder=10)
plt.scatter([i for i in num_nodes], 1/time_per_cct(df_ondemand), label="$\\textrm{on-demand}$", marker="o", color=colors[2], ec="k", zorder=9)
plt.plot([i for i in num_nodes],  func(df_batched["num_nodes"], *popt_batch), ':', linewidth=3, label=f"$f(x) = {popt_batch[0]:{4}.{3}}x + {popt_batch[1]:{4}.{3}}$", alpha=0.95, color=colors[1], zorder=1)
plt.plot([i for i in num_nodes],  func(df_ondemand["num_nodes"], *popt_ondemand), ':', linewidth=3,label=f"$f(x) = {popt_ondemand[0]:{4}.{3}}x + {popt_ondemand[1]:{4}.{3}}$", alpha=0.95, color=colors[3], zorder=0)

xticks = list(num_nodes)
xticks.remove(2)

plt.xticks(ticks = xticks, labels=[f"${i}$" for i in xticks])

plt.xlabel("$\\textrm{nodes}$")
plt.ylabel("$\\textrm{circuit per second}~(s^{-1})$")

plt.ylim([-0.5, 90])
plt.xlim([0, 70])

plt.legend(loc='best', fontsize=12, ncol=2, fancybox=True, shadow=True)
plt.tight_layout()

In [ ]:
plt.savefig("lightning_gpu_task_based_qcut_45k_scaling_cct_seconds.pdf", dpi=300)

In [ ]:
plt.tight_layout()

# 2x1 plot of above figures

In [ ]:
cmap_alt = matplotlib.colormaps['tab20b']
colors_alt = cmap_alt.colors

In [ ]:
f, axs = plt.subplots(2,1, sharex=True, squeeze=True)
plt.rcParams.update({'font.size': 16})


axs[0].plot(df_batched["num_gpus"], df_batched["frag_exec_time"], linewidth=3, label="batched", marker="o", color=colors_alt[2], mec="k", mfc="w")
axs[0].plot(df_ondemand["num_gpus"], df_ondemand["frag_exec_time"], linewidth=3, label="on-demand", marker="o", color=colors_alt[14], mec="k", mfc="w")
axs[0].set_yscale("log")
axs[0].set_xscale("log", base=2)
axs[0].set_xticks(df_ondemand["num_gpus"], labels=[f"${{{i}}}$" for i in df_ondemand["num_gpus"]])
axs[0].set_ylabel("$t~\\textrm{(s)}$")

popt_batch, pcov_batch = curve_fit(func, np.array(df_batched["num_gpus"]),  1/time_per_cct(df_batched) )
popt_ondemand, pcov_ondemand = curve_fit(func, np.array(df_ondemand["num_gpus"]), 1/time_per_cct(df_ondemand) )
num_gpus = df_batched["num_gpus"]

axs[1].plot([i for i in num_gpus], 1/time_per_cct(df_batched), linewidth=3, label="$\\textrm{batched}$",  marker="o", color=colors_alt[2], mec="k", mfc="w", zorder=1)
axs[1].plot([i for i in num_gpus], 1/time_per_cct(df_ondemand), linewidth=3, label="$\\textrm{on-demand}$", marker="o", color=colors_alt[14], mec="k", mfc="w", zorder=0)
axs[1].plot([i for i in num_gpus],  func(df_batched["num_gpus"], *popt_batch), ':', linewidth=3, label=f"$f(x) = {popt_batch[0]:{4}.{3}}x + {popt_batch[1]:{4}.{3}}$", alpha=0.95, color=colors_alt[0], zorder=10)
axs[1].plot([i for i in num_gpus],  func(df_ondemand["num_gpus"], *popt_ondemand), ':', linewidth=3,label=f"$f(x) = {popt_ondemand[0]:{4}.{3}}x + {popt_ondemand[1]:{4}.{3}}$", alpha=0.95, color=colors_alt[12], zorder=9)

xticks = list(num_gpus)

axs[1].set_xticks(ticks = xticks, labels=[f"${i}$" for i in xticks])
axs[1].set_yticks(ticks = range(0,100,25), labels=[f"${i}$" for i in range(0,100,25)])

axs[1].set_xlabel("$\\textrm{GPUs}$")
axs[1].set_ylabel("$\\textrm{circ./sec.}~(s^{-1})$")

axs[1].set_ylim([-3.0, 90])
axs[1].set_xlim([-2, 290])

In [ ]:
plt.tight_layout()

In [ ]:
plt.subplots_adjust(wspace=0.35, hspace=0.1)

In [ ]:
axs[1].legend(loc='lower center', fontsize=14, bbox_to_anchor=(0.45, -1.0), ncol=2, fancybox=True, shadow=True, labelspacing=0.5)

In [ ]:
plt.savefig("lightning_gpu_task_based_qcut_45k_scaling_cct_cu12.pdf", dpi=300)